In [8]:
import mysql.connector as db
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
import numpy as np

class db_communication:
    """
    ____________________________________________________________________________________________
    Benötigte Python-Module:
    - mysql
    - mysql-connector
    - pandas
    - numpy
    ____________________________________________________________________________________________
    Bietet eine API zur Kommnunikation mit der Fahrrad-Shop-Datenbank.
    Auf Basis des mysql-Moduls wird mit einer MariaDB kommuniziert.
    
    Durch print()-Ausgaben wird der Erfolg oder Misserfolg bei einem Funktionsaufruf ausgegeben.
    
    Die Klasse ist wie folgt zu nutzen:
    my_db = db_communication()
    -----
    my_db.user_func_xyz(args)
    -----
    my_db.close()
    """
    def __init__(self, config = None):
        """
        Default (config = None) wird die Verbindung zur Standard-Datenbank hergestellt.
        Alternativ kann ein config-dict übergeben werden
        config = {
            'user': '',
            'password': '',
            'host': '',
            'port': '',
            'database': ''
        }
        """
        self.__db = self.connect(config)
        if self.__db:
            self.__cursor = self.__db.cursor()
            print("Successfully connected.")
        else:
            print("... no connection ...")

    def connect(self, config=None) -> db.MySQLConnection:    
        """
        Stellt die Verbindung zur Datenbank her und nutzt die übergeben Parameter config.
        Return: 'db.MySQLConnection'.
        """
        if not config:
            config = {
            'user': 'root',
            'password': 'FDS-apm1',
            'host': 'min-ifm-xdm.ad.fh-bielefeld.de',
            'port': '3306',
            'database': 'NeueDB'
            }

        try:
            return mysql.connector.connect(**config)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Something is wrong with your user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Database does not exist")
            else:
                print(err)
        else:
            print("Not connected")
            
    def close(self):
        """ 
        Schließt die Verbindung zur Datenbank.
        """
        if self.__db:
            self.__db.close()
            print("Connection closed")
        else:
            pass
    
    def read_tables(self) -> list:
        """
        Liest die Tabellen der Datenbank aus und gibt diese in Listen-Form zurück.
        """
        if self.__db:
            self.__cursor.execute("SHOW TABLES")

            tables = list()
            for x in self.__cursor:
                tables.append(x)

            return tables
        else:
            pass
        
    def insert_data(self, table_name, data_as_dict):
        """
        Der Tabelle 'table_name' wird eine neue Zeile hinzugefügt. 
        Der Inhalt der Zeile wird aus dem 'data_as_dict' erstellt. 
        Die Spalentennamen sind die 'keys' und die Zellenwerte die 'values' des Dictionaries.
        """
        try:
            sql = "INSERT INTO `" + table_name + "` ("
            values = ""
            for key in data_as_dict:
                sql += "`" + str(key) + "`, "
                if data_as_dict[key]: 
                    values += "'" + str(data_as_dict[key]) + "', "
                else:
                    values += "NULL , "
            sql = sql[:len(sql)-2]
            values = values[:len(values)-2]
            sql += ") VALUES (" + values + ");"
            
            print(sql)
            self.__cursor.execute(sql)
            self.__db.commit()
            return True
        except:
           # Rollback in case there is any error
            self.__db.rollback()
            print("Rollback, something went wrong.")
            return False
    
    def get_table(self, table_name, key = None, column = None) -> pd.DataFrame:
        """
        Die gewünschte Tabelle 'table_name' wird als pd.DataFrame zurückgegeben.
        """
        if key and column:
            pass # was war hier nochmal?
        elif column:
            self.__cursor.execute("SELECT {} FROM `{}`".format(column, table_name))
            rows = []
            for y in self.__cursor:
                rows.append(y)
            return pd.DataFrame(np.array(rows), columns = [column])
        elif key:
            pass # was war hier nochmal?
        else:
            self.__cursor.execute("SHOW COLUMNS FROM `{}`".format(table_name))
            columns = []
            for y in self.__cursor:
                columns.append(y[0])
            self.__cursor.execute("SELECT * FROM `{}`".format(table_name))
            rows = []
            for y in self.__cursor:
                rows.append(y)
            try: 
                df = pd.DataFrame(np.array(rows), columns=columns)
                return df
            except ValueError:
                return None
        
    def drop_table(self, name) -> bool:
        """
        Die Tabelle name wird gelöscht, allerdings nur unter der Bedingung, 
        dass die Tabelle keine Abhängigkeiten zu anderen Tabellen hat.
        """
        sql = "DROP TABLE %s" %str(name)
        try:
            # Execute the SQL command
            self.__cursor.execute(sql)
            # Commit your changes in the database
            self.__db.commit()
            print("Drop table: ", str(name))
        except:
           # Rollback in case there is any error
            self.__db.rollback()
            print("Rollback, something went wrong.")
            return False
        
        return True

    def show_columns(self, table_name):
        """
        Gibt die Spalten der gewünschten Tabelle über die print-Ausgabe aus.
        """
        tables = self.read_tables()
        if not any(name in s for s in tables):
            return self.__cursor.execute("SHOW COLUMNS FROM `{}`".format(x))
        else:
            print("Table", name, "already exists.")
            return False
        
    def storno(self, key):
        """
        Datenbank spezifische Funktion.
        Eine existierende Bestellung wird storniert, identifiziert an dem 'key' == 'Bestellung.id'. 
        Die Spalte "Storniert" wird auf True gesetzt.
        """
        
        sql = "UPDATE `Bestellungen` SET `Storniert`='1' WHERE `Bestellungen.id`= '" + key + "';"

        try:
            # Execute the SQL command
            self.__cursor.execute(sql)
            # Commit your changes in the database
            self.__db.commit()
            print("Update Bestellung:", str(key))
        except:
           # Rollback in case there is any error
            self.__db.rollback()
            print("Rollback, something went wrong.")
            return False
    
    def create_table(self, name, rows) -> bool:
        """
        Erstellt eine Tabelle mit dem Namen 'name' und den Spalten 'rows'.
        'rows'-Elemente müssen im MariaDB-Format die Einstellungen enthalten. 
        Bsp.1: 
        `Bauteil.id` INT NOT NULL AUTO_INCREMENT
        Bsp.2:
        FOREIGN KEY (`Baugruppe`) REFERENCES `Baugruppen`(`Baugruppe`)
        """
        tables = self.read_tables()
        if not any(name in s for s in tables):
            sql = "CREATE TABLE `" + name + "` ("
            for row in rows:     
                if rows.index(row) == (len(rows)-1):
                    sql += " " + row
                else:
                    sql += " " + row + ","
            sql += ")"
            print(sql)
            self.__cursor.execute(sql)
            print("Created table: ", name)
            return True
        else:
            print("Table", name, "already exists.")
            return False
        
    def reset_all_tables(self, are_u_sure = False):
        """
        Funktion um die Tabellen der Datenbank zu löschen und neu und leer wieder zu initialiseren.
        Die Variable 'are_u_sure' muss auf True gesetzt sein, um den Vorgang durchzuführen
        Der Aufbau der Tabellen ist fest niedergeschrieben und hat Abhängigkeiten wie FOREIGN KEY untereinander.
        Es ist auf die richtige Reihenfolge zu achten!
        Aus diesem Grund wird das Nutzen dieser Funktion empfohlen, um die Tabellen der Fahrrad-Shop-Datenbank zu löschen
        """
        config_kunde = ["`KundenNr` INTEGER NOT NULL AUTO_INCREMENT",
                        "`Name` CHAR(30) NOT NULL",
                        "`Vorname` CHAR(30) NOT NULL",
                        "`Mail` CHAR(60)",
                        "PRIMARY KEY (`KundenNr`)"]

        config_auftrag = [ "`AuftrNr` INTEGER NOT NULL AUTO_INCREMENT",
                           "`Bestelldatum` DATE",
                           "`Lieferdatum` DATE",
                           "`KundenNr` INTEGER NOT NULL",
                           "`Gesamtpreis` NUMERIC(7,2)",
                           "PRIMARY KEY (`AuftrNr`)",
                           "FOREIGN KEY (`KundenNr`) REFERENCES `Kunde`(`KundenNr`)"]

        config_konfiguration = ["`KonfigNr` INTEGER NOT NULL AUTO_INCREMENT",
                                "`AuftragNr` INTEGER NOT NULL",
                                "`MerkmalNr` INTEGER NOT NULL",
                                "PRIMARY KEY (`KonfigNr`)",
                                "FOREIGN KEY (`AuftragNr`) REFERENCES `Auftrag`(`AuftrNr`)",
                                "FOREIGN KEY (`MerkmalNr`) REFERENCES `Merkmale`(`MerkmalNr`)"]

        config_merkmalcluster = [ "`ClusterNr` INTEGER NOT NULL AUTO_INCREMENT",
                                  "`Merkmalsbezeichnung`CHAR(35)",
                                  "PRIMARY KEY (`ClusterNr`)"]

        config_merkmale = [ "`MerkmalNr` INTEGER NOT NULL AUTO_INCREMENT",
                            "`Merkmalcluster` INTEGER NOT NULL",
                            " Bezeichnung CHAR(35) NOT NULL",
                            "`Preis` NUMERIC(7,2)",
                            "`Montagezeit` INTEGER",
                            "`Ausstattung` CHAR(35)",
                            "PRIMARY KEY (`MerkmalNr`)",
                            "FOREIGN KEY (`Merkmalcluster`) REFERENCES `Merkmalcluster`(`ClusterNr`)"]

        config_einzelteil = [ "`EinzelteilNr` INTEGER NOT NULL AUTO_INCREMENT",
                              " Bezeichnung CHAR(35) NOT NULL",
                              "`Preis` NUMERIC(7,2)",
                              "PRIMARY KEY (`EinzelteilNr`)"]


        config_stuecklistenteil = [ "`MerkmalNr` INTEGER",
                                    "`EinzelteilNr` INTEGER",
                                    "`Anzahl` INTEGER",
                                    "FOREIGN KEY (`MerkmalNr`) REFERENCES `Merkmale`(`MerkmalNr`)",
                                    "FOREIGN KEY (`EinzelteilNr`) REFERENCES `Einzelteil`(`EinzelteilNr`)",
                                    "PRIMARY KEY (`MerkmalNr`,`EinzelteilNr`)"]
        
        if are_u_sure:
            list_names_tables = ["Kunde","Auftrag","Merkmalcluster","Merkmale","Einzelteil","Konfiguration","Stuecklistenteil"]
            list_config_tables=[config_kunde,config_auftrag,config_merkmalcluster,config_merkmale,config_einzelteil,config_konfiguration,config_stuecklistenteil]
            table_names = list_config_tables[::-1]
            for t in table_names:
                self.drop_table(t)

        for idx in range(len(list_config_tables)):
            self.create_table(list_names_tables[idx],list_config_tables[idx])

In [9]:
#Blanko DB
my_db = db_communication()
my_db.reset_all_tables(True)
my_db.close()

Successfully connected.
Rollback, something went wrong.
Rollback, something went wrong.
Rollback, something went wrong.
Rollback, something went wrong.
Rollback, something went wrong.
Rollback, something went wrong.
Rollback, something went wrong.
Table Kunde already exists.
Table Auftrag already exists.
Table Merkmalcluster already exists.
Table Merkmale already exists.
Table Einzelteil already exists.
Table Konfiguration already exists.
CREATE TABLE `Stuecklistenteil` ( `MerkmalNr` INTEGER, `EinzelteilNr` INTEGER, `Anzahl` INTEGER, FOREIGN KEY (`MerkmalNr`) REFERENCES `Merkmale`(`MerkmalNr`), FOREIGN KEY (`EinzelteilNr`) REFERENCES `Einzelteil`(`EinzelteilNr`), PRIMARY KEY (`MerkmalNr`,`EinzelteilNr`))
Created table:  Stuecklistenteil
Connection closed


In [10]:
#Fuelle DB 
kunde = [{'KundenNr' : '1','Name' : 'Kraus','Vorname':'Maria','Mail' : 'MaKrau@radelwelt.com'}]
my_db = db_communication()
for i in range(len(kunde)):
    my_db.insert_data("Kunde", kunde[i])
my_db.close()

Successfully connected.
INSERT INTO `Kunde` (`KundenNr`, `Name`, `Vorname`, `Mail`) VALUES ('1', 'Kraus', 'Maria', 'MaKrau@radelwelt.com');
Connection closed


In [11]:
auftrag = [{'AuftrNr':'1','Bestelldatum':'2020-04-20','Lieferdatum':'2020-06-02','Gesamtpreis': '1313.54','KundenNr':'1'}]

my_db = db_communication()
for i in range(len(auftrag)):
    my_db.insert_data("Auftrag", auftrag[i])
my_db.close()

Successfully connected.
INSERT INTO `Auftrag` (`AuftrNr`, `Bestelldatum`, `Lieferdatum`, `Gesamtpreis`, `KundenNr`) VALUES ('1', '2020-04-20', '2020-06-02', '1313.54', '1');
Connection closed


In [12]:
merkmalcluster  = [ {'ClusterNr':'1','Merkmalsbezeichnung':'Rahmen'},
                    {'ClusterNr':'2','Merkmalsbezeichnung':'Gabel'},
                    {'ClusterNr':'3','Merkmalsbezeichnung':'Lackierung'},
                    {'ClusterNr':'4','Merkmalsbezeichnung':'Lenker'},
                    {'ClusterNr':'5','Merkmalsbezeichnung':'Felgen'},
                    {'ClusterNr':'6','Merkmalsbezeichnung':'Radbereifung'},
                    {'ClusterNr':'7','Merkmalsbezeichnung':'Gangschaltung'},
                    {'ClusterNr':'8','Merkmalsbezeichnung':'Sattel'},
                    {'ClusterNr':'9','Merkmalsbezeichnung':'Griffe'},
                    {'ClusterNr':'10','Merkmalsbezeichnung':'Bremsen'}]

my_db = db_communication()
for i in range(len(merkmalcluster)):
    my_db.insert_data("Merkmalcluster", merkmalcluster[i])
my_db.close()

Successfully connected.
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('1', 'Rahmen');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('2', 'Gabel');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('3', 'Lackierung');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('4', 'Lenker');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('5', 'Felgen');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('6', 'Radbereifung');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('7', 'Gangschaltung');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('8', 'Sattel');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('9', 'Griffe');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('10', 'Bremsen');
Connection closed


In [13]:
merkmale = [ 
    #Rahmen
    { 'MerkmalNr':'100001','Merkmalcluster':'1','Bezeichnung': 'FH100','Preis':'499.00','Montagezeit': '2','Ausstattung':'City'},
    { 'MerkmalNr':'100002','Merkmalcluster':'1','Bezeichnung': 'S100','Preis':'649.99','Montagezeit': '2','Ausstattung':'Sport'},
    { 'MerkmalNr':'100003','Merkmalcluster':'1','Bezeichnung': 'Trofeo 5','Preis':'599.00','Montagezeit': '2','Ausstattung':'Rennen'},
    
    { 'MerkmalNr':'199999','Merkmalcluster':'1','Bezeichnung': 'Kein Rahmen','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Gabel
    { 'MerkmalNr':'200001','Merkmalcluster':'2','Bezeichnung': 'Aluminium Citystoßdämpfer','Preis':'9.99','Montagezeit': '1','Ausstattung':'City'},
    { 'MerkmalNr':'200002','Merkmalcluster':'2','Bezeichnung': 'Carbon Öldruckstoßdämpfer','Preis':'49.00','Montagezeit': '2','Ausstattung':'Sport'},
    { 'MerkmalNr':'200003','Merkmalcluster':'2','Bezeichnung': 'Carbon starr','Preis':'29.00','Montagezeit': '1','Ausstattung':'Rennen'},
    
    { 'MerkmalNr':'299999','Merkmalcluster':'2','Bezeichnung': 'Keine Gabel','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Lackierung/ Farbe
    { 'MerkmalNr':'300001','Merkmalcluster':'3','Bezeichnung': 'Schwarz uni','Preis':'10.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'300002','Merkmalcluster':'3','Bezeichnung': 'Weiß uni','Preis':'15.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'300003','Merkmalcluster':'3','Bezeichnung': 'Saphirschwarz metallic','Preis':'29.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'300004','Merkmalcluster':'3','Bezeichnung': 'Mineralweiß metallic','Preis':'29.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'300005','Merkmalcluster':'3','Bezeichnung': 'Phytonicblau','Preis':'50.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'300006','Merkmalcluster':'3','Bezeichnung': 'Aventurinrot II metallic','Preis':'65.00','Montagezeit': '2','Ausstattung':'NULL'},
    { 'MerkmalNr':'300007','Merkmalcluster':'3','Bezeichnung': 'Frozen Bluestone metallic','Preis':'70.00','Montagezeit': '2','Ausstattung':'NULL'},
    
    { 'MerkmalNr':'399999','Merkmalcluster':'3','Bezeichnung': 'Keine Lackierung','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Lenker
    { 'MerkmalNr':'400001','Merkmalcluster':'4','Bezeichnung': 'Retro Lenker','Preis':'89.99','Montagezeit': '1','Ausstattung':'Sport'},
    { 'MerkmalNr':'400002','Merkmalcluster':'4','Bezeichnung': 'Classic Lenker','Preis':'59.49','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'400003','Merkmalcluster':'4','Bezeichnung': 'Rennradlenker','Preis':'99.99','Montagezeit': '1','Ausstattung':'Rennen'},
    
    { 'MerkmalNr':'499999','Merkmalcluster':'4','Bezeichnung': 'Kein Lenker','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Felgen
    { 'MerkmalNr':'500001','Merkmalcluster':'5','Bezeichnung': 'Aluminium Felgen','Preis':'10.00','Montagezeit': '3','Ausstattung':'NULL'},
    { 'MerkmalNr':'500002','Merkmalcluster':'5','Bezeichnung': 'Redondo Rennradfelgen','Preis':'99.99','Montagezeit': '4','Ausstattung':'Rennen'},
    { 'MerkmalNr':'500003','Merkmalcluster':'5','Bezeichnung': 'Spengle Carbonfelgen','Preis':'79.99','Montagezeit': '2','Ausstattung':'NULL'},
    { 'MerkmalNr':'500004','Merkmalcluster':'5','Bezeichnung': 'Taylor Sportfelgen','Preis':'39.99','Montagezeit': '5','Ausstattung':'Sport'},
    { 'MerkmalNr':'500005','Merkmalcluster':'5','Bezeichnung': 'Taylor Cityfelgen','Preis':'69.99','Montagezeit': '5','Ausstattung':'City'},
    
    { 'MerkmalNr':'599999','Merkmalcluster':'5','Bezeichnung': 'Keine Felge','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Radbereifung
    { 'MerkmalNr':'600001','Merkmalcluster':'6','Bezeichnung': 'Bereifung Schwalbe City','Preis':'15.00','Montagezeit': '2','Ausstattung':'City'},
    { 'MerkmalNr':'600002','Merkmalcluster':'6','Bezeichnung': 'Bereifung Schwalbe Sport','Preis':'20.00','Montagezeit': '1','Ausstattung':'Sport'},
    { 'MerkmalNr':'600003','Merkmalcluster':'6','Bezeichnung': 'Bereifung Schwalbe Marathon','Preis':'35.00','Montagezeit': '4','Ausstattung':'Rennen'},
    
    { 'MerkmalNr':'699999','Merkmalcluster':'6','Bezeichnung': 'Keine Reifen','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Gangschaltung
    { 'MerkmalNr':'700001','Merkmalcluster':'7','Bezeichnung': '3 Gang Nexus RT','Preis':'100.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'700002','Merkmalcluster':'7','Bezeichnung': '7 Gang Nexus RT','Preis':'110.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'700003','Merkmalcluster':'7','Bezeichnung': '9 Gang Nexus RT','Preis':'150.00','Montagezeit': '1','Ausstattung':'NULL'},
    
    { 'MerkmalNr':'799999','Merkmalcluster':'7','Bezeichnung': 'Keine Schaltung','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Sattel
    { 'MerkmalNr':'800001','Merkmalcluster':'8','Bezeichnung': 'Asvert Cityradsattel','Preis':'10.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'800002','Merkmalcluster':'8','Bezeichnung': 'Asvert Comfortsattel','Preis':'39.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'800003','Merkmalcluster':'8','Bezeichnung': 'Avert Rennradsattel','Preis':'59.00','Montagezeit': '1','Ausstattung':'Rennen'},
    
    { 'MerkmalNr':'899999','Merkmalcluster':'8','Bezeichnung': 'Kein Sattel','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Griffe
    { 'MerkmalNr':'900001','Merkmalcluster':'9','Bezeichnung': 'Girff schwarz Weichgummi','Preis':'11.99','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'900002','Merkmalcluster':'9','Bezeichnung': 'Griff schwarz Hartgummi','Preis':'11.99','Montagezeit': '1','Ausstattung':'Rennen'},
    { 'MerkmalNr':'900003','Merkmalcluster':'9','Bezeichnung': 'Griff schwarz Kautschuk','Preis':'10.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'900003','Merkmalcluster':'9','Bezeichnung': 'Griff braun Leder','Preis':'19.99','Montagezeit': '1','Ausstattung':'City'},
    
    { 'MerkmalNr':'999999','Merkmalcluster':'9','Bezeichnung': 'Keine Griffe','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Bremsen
    { 'MerkmalNr':'1000001','Merkmalcluster':'10','Bezeichnung': 'Shimano V-Brake','Preis':'10.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'1000002','Merkmalcluster':'10','Bezeichnung': 'Clarks Scheibenbremse','Preis':'69.99','Montagezeit': '2','Ausstattung':'Sport'},
    { 'MerkmalNr':'1000003','Merkmalcluster':'10','Bezeichnung': 'Shimano GRB Scheibenbremse','Preis':'99.99','Montagezeit': '2','Ausstattung':'NULL'},
    { 'MerkmalNr':'1000004','Merkmalcluster':'10','Bezeichnung': 'Fagura Felgenbremse','Preis':'39.99','Montagezeit': '1','Ausstattung':'NULL'},
    
    { 'MerkmalNr':'1099999','Merkmalcluster':'10','Bezeichnung': 'Keine Bremse','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
]
my_db = db_communication()
for i in range(len(merkmale)):
    my_db.insert_data("Merkmale", merkmale[i])
my_db.close()

Successfully connected.
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`, `Montagezeit`, `Ausstattung`) VALUES ('100001', '1', 'FH100', '499.00', '2', 'City');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`, `Montagezeit`, `Ausstattung`) VALUES ('100002', '1', 'S100', '649.99', '2', 'Sport');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`, `Montagezeit`, `Ausstattung`) VALUES ('100003', '1', 'Trofeo 5', '599.00', '2', 'Rennen');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`, `Montagezeit`, `Ausstattung`) VALUES ('199999', '1', 'Kein Rahmen', '0.00', '0', 'NULL');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`, `Montagezeit`, `Ausstattung`) VALUES ('200001', '2', 'Aluminium Citystoßdämpfer', '9.99', '1', 'City');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`, `Montagezeit`, `Ausstattung`) VALUES ('200002

In [17]:
einzelteile = [{ 'EinzelteilNr':'1001','Bezeichnung': 'Rückstrahler','Preis':'5.00'},
{ 'EinzelteilNr':'2001','Bezeichnung': 'Bremsgriff','Preis':'9.99'},
{ 'EinzelteilNr':'4001','Bezeichnung': 'Zahnkranz','Preis':'50.00'},
{ 'EinzelteilNr':'4002','Bezeichnung': 'Kette','Preis':'9.00'},
{ 'EinzelteilNr':'4003','Bezeichnung': 'Kettenblätter','Preis':'3.99'},
{ 'EinzelteilNr':'4004','Bezeichnung': 'Pedale','Preis':'8.99'},
{ 'EinzelteilNr':'4005','Bezeichnung': 'Innenlager','Preis':'3.99'},
{ 'EinzelteilNr':'4006','Bezeichnung': 'Tretkurbel','Preis':'3.99'},
{ 'EinzelteilNr':'5001','Bezeichnung': 'Steuersatz','Preis':'80.00'},
{ 'EinzelteilNr':'5002','Bezeichnung': 'Steuerrohr','Preis':'12.99'},
{ 'EinzelteilNr':'5003','Bezeichnung': 'Gummigriffe','Preis':'0.99'},
{ 'EinzelteilNr':'5004','Bezeichnung': 'Klingel','Preis':'7.99'},
{ 'EinzelteilNr':'6001','Bezeichnung': 'Sitzrohr','Preis':'4.50'},
{ 'EinzelteilNr':'6002','Bezeichnung': 'Sitzstrebe','Preis':'17.50'},
{ 'EinzelteilNr':'7001','Bezeichnung': 'Ventil','Preis':'6.99'},
{ 'EinzelteilNr':'7002','Bezeichnung': 'Schlauch','Preis':'2.99'},
{ 'EinzelteilNr':'7003','Bezeichnung': 'Schutzblech','Preis':'12.99'},
{ 'EinzelteilNr':'8001','Bezeichnung': 'Sattelstütze','Preis':'15.00'},
{ 'EinzelteilNr':'9001','Bezeichnung': 'Schraube','Preis':'0.25'},
{ 'EinzelteilNr':'9002','Bezeichnung': 'Mutter','Preis':'0.50'},
{ 'EinzelteilNr':'9003','Bezeichnung': 'Bremskabel 100 cm','Preis':'1.50'},
{ 'EinzelteilNr':'9004','Bezeichnung': 'Stromkabel 50 cm' ,'Preis':'2.50'},
{ 'EinzelteilNr':'9005','Bezeichnung': 'Lackierung','Preis':'25.50'}]
my_db = db_communication()
for i in range(len(einzelteile)):
    my_db.insert_data("Einzelteil", einzelteile[i])
my_db.close()

Successfully connected.
INSERT INTO `Einzelteil` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('1001', 'Rückstrahler', '5.00');
INSERT INTO `Einzelteil` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('2001', 'Bremsgriff', '9.99');
INSERT INTO `Einzelteil` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('4001', 'Zahnkranz', '50.00');
INSERT INTO `Einzelteil` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('4002', 'Kette', '9.00');
INSERT INTO `Einzelteil` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('4003', 'Kettenblätter', '3.99');
INSERT INTO `Einzelteil` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('4004', 'Pedale', '8.99');
INSERT INTO `Einzelteil` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('4005', 'Innenlager', '3.99');
INSERT INTO `Einzelteil` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('4006', 'Tretkurbel', '3.99');
INSERT INTO `Einzelteil` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('5001', 'Steuersatz', '80.00');
INSERT INTO `Einzelteil` (`Einzelt

In [19]:
konfiguration =[{'KonfigNr':'101','AuftragNr':'1','MerkmalNr':'100002'},
               {'KonfigNr':'102','AuftragNr':'1','MerkmalNr':'200002'},
               {'KonfigNr':'103','AuftragNr':'1','MerkmalNr':'300001'},
               {'KonfigNr':'104','AuftragNr':'1','MerkmalNr':'400003'},
               {'KonfigNr':'105','AuftragNr':'1','MerkmalNr':'500002'},
               {'KonfigNr':'106','AuftragNr':'1','MerkmalNr':'600002'},
               {'KonfigNr':'107','AuftragNr':'1','MerkmalNr':'700002'},
               {'KonfigNr':'108','AuftragNr':'1','MerkmalNr':'800004'}]
                
my_db = db_communication()
for i in range(len(konfigurator)):
    my_db.insert_data("Konfiguration", konfiguration[i])
my_db.close()

Successfully connected.
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('101', '1', '100002');
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('102', '1', '200002');
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('103', '1', '300001');
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('104', '1', '400003');
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('105', '1', '500002');
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('106', '1', '600002');
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('107', '1', '700002');
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('108', '1', '800004');
Rollback, something went wrong.
Connection closed


In [23]:
#TODO
stuecklistenteil = [{'MerkmalNr': '100001','EinzelteilNr':'9001','Anzahl':'3'},
               {'MerkmalNr': '100001','EinzelteilNr':'9002','Anzahl':'3'},
               {'MerkmalNr': '100001','EinzelteilNr':'9004','Anzahl':'45'},
               {'MerkmalNr': '100002','EinzelteilNr':'9001','Anzahl':'4'},
               {'MerkmalNr': '100002','EinzelteilNr':'9002','Anzahl':'4'},
               {'MerkmalNr': '100002','EinzelteilNr':'9004','Anzahl':'55'},
               {'MerkmalNr': '200001','EinzelteilNr':'2001','Anzahl':'2'},
               {'MerkmalNr': '200001','EinzelteilNr':'9001','Anzahl':'4'},
               {'MerkmalNr': '200001','EinzelteilNr':'9002','Anzahl':'4'},
               {'MerkmalNr': '200001','EinzelteilNr':'9003','Anzahl':'35'},
               {'MerkmalNr': '200002','EinzelteilNr':'2001','Anzahl':'2'},
               {'MerkmalNr': '200002','EinzelteilNr':'9001','Anzahl':'5'},
               {'MerkmalNr': '200002','EinzelteilNr':'9002','Anzahl':'5'},
               {'MerkmalNr': '200002','EinzelteilNr':'9003','Anzahl':'40'},
               {'MerkmalNr': '200003','EinzelteilNr':'2001','Anzahl':'2'},
               {'MerkmalNr': '200003','EinzelteilNr':'9001','Anzahl':'6'},
               {'MerkmalNr': '200003','EinzelteilNr':'9002','Anzahl':'6'},
               {'MerkmalNr': '200003','EinzelteilNr':'9003','Anzahl':'42'},
               {'MerkmalNr': '300001','EinzelteilNr':'9001','Anzahl':'3'},
               {'MerkmalNr': '300001','EinzelteilNr':'9002','Anzahl':'3'},
               {'MerkmalNr': '300002','EinzelteilNr':'9001','Anzahl':'3'},
               {'MerkmalNr': '300002','EinzelteilNr':'9002','Anzahl':'3'},
               {'MerkmalNr': '300003','EinzelteilNr':'9001','Anzahl':'3'},
               {'MerkmalNr': '300003','EinzelteilNr':'9002','Anzahl':'3'},
               {'MerkmalNr': '300004','EinzelteilNr':'9001','Anzahl':'3'},
               {'MerkmalNr': '300004','EinzelteilNr':'9002','Anzahl':'3'},
               {'MerkmalNr': '300005','EinzelteilNr':'9001','Anzahl':'3'},
               {'MerkmalNr': '300005','EinzelteilNr':'9002','Anzahl':'3'},
               {'MerkmalNr': '400001','EinzelteilNr':'4001','Anzahl':'1'},
               {'MerkmalNr': '400001','EinzelteilNr':'4002','Anzahl':'1'},
               {'MerkmalNr': '400001','EinzelteilNr':'4003','Anzahl':'1'},
               {'MerkmalNr': '400001','EinzelteilNr':'4004','Anzahl':'2'},
               {'MerkmalNr': '400001','EinzelteilNr':'4005','Anzahl':'1'},
               {'MerkmalNr': '400001','EinzelteilNr':'4006','Anzahl':'2'},
               {'MerkmalNr': '400001','EinzelteilNr':'9001','Anzahl':'17'},
               {'MerkmalNr': '400001','EinzelteilNr':'9002','Anzahl':'17'},               
               {'MerkmalNr': '400002','EinzelteilNr':'4001','Anzahl':'1'},
               {'MerkmalNr': '400002','EinzelteilNr':'4002','Anzahl':'1'},
               {'MerkmalNr': '400002','EinzelteilNr':'4003','Anzahl':'1'},
               {'MerkmalNr': '400002','EinzelteilNr':'4004','Anzahl':'2'},
               {'MerkmalNr': '400002','EinzelteilNr':'4005','Anzahl':'1'},
               {'MerkmalNr': '400002','EinzelteilNr':'4006','Anzahl':'2'},
               {'MerkmalNr': '400002','EinzelteilNr':'9001','Anzahl':'17'},
               {'MerkmalNr': '400002','EinzelteilNr':'9002','Anzahl':'17'},               
               {'MerkmalNr': '400003','EinzelteilNr':'4001','Anzahl':'1'},
               {'MerkmalNr': '400003','EinzelteilNr':'4002','Anzahl':'1'},
               {'MerkmalNr': '400003','EinzelteilNr':'4003','Anzahl':'1'},
               {'MerkmalNr': '400003','EinzelteilNr':'4004','Anzahl':'2'},
               {'MerkmalNr': '400003','EinzelteilNr':'4005','Anzahl':'1'},
               {'MerkmalNr': '400003','EinzelteilNr':'4006','Anzahl':'2'},
               {'MerkmalNr': '400003','EinzelteilNr':'9001','Anzahl':'17'},
               {'MerkmalNr': '400003','EinzelteilNr':'9002','Anzahl':'17'},               
               {'MerkmalNr': '500001','EinzelteilNr':'5001','Anzahl':'1'},
               {'MerkmalNr': '500001','EinzelteilNr':'5002','Anzahl':'1'},
               {'MerkmalNr': '500001','EinzelteilNr':'5003','Anzahl':'2'},
               {'MerkmalNr': '500001','EinzelteilNr':'5004','Anzahl':'1'},
               {'MerkmalNr': '500001','EinzelteilNr':'9001','Anzahl':'5'},
               {'MerkmalNr': '500001','EinzelteilNr':'9002','Anzahl':'5'},               
               {'MerkmalNr': '500002','EinzelteilNr':'5001','Anzahl':'1'},
               {'MerkmalNr': '500002','EinzelteilNr':'5002','Anzahl':'1'},
               {'MerkmalNr': '500002','EinzelteilNr':'5003','Anzahl':'2'},
               {'MerkmalNr': '500002','EinzelteilNr':'5004','Anzahl':'1'},
               {'MerkmalNr': '500002','EinzelteilNr':'9001','Anzahl':'5'},
               {'MerkmalNr': '500002','EinzelteilNr':'9002','Anzahl':'5'},              
               {'MerkmalNr': '500003','EinzelteilNr':'5001','Anzahl':'1'},
               {'MerkmalNr': '500003','EinzelteilNr':'5002','Anzahl':'1'},
               {'MerkmalNr': '500003','EinzelteilNr':'5003','Anzahl':'2'},
               {'MerkmalNr': '500003','EinzelteilNr':'5004','Anzahl':'1'},
               {'MerkmalNr': '500003','EinzelteilNr':'9001','Anzahl':'5'},
               {'MerkmalNr': '500003','EinzelteilNr':'9002','Anzahl':'5'},            
               {'MerkmalNr': '600001','EinzelteilNr':'6001','Anzahl':'1'},
               {'MerkmalNr': '600001','EinzelteilNr':'6002','Anzahl':'1'},
               {'MerkmalNr': '600001','EinzelteilNr':'9001','Anzahl':'8'},
               {'MerkmalNr': '600001','EinzelteilNr':'9002','Anzahl':'8'},
               {'MerkmalNr': '600001','EinzelteilNr':'9005','Anzahl':'1'},
               {'MerkmalNr': '600002','EinzelteilNr':'6001','Anzahl':'1'},
               {'MerkmalNr': '600002','EinzelteilNr':'6002','Anzahl':'1'},
               {'MerkmalNr': '600002','EinzelteilNr':'9001','Anzahl':'8'},
               {'MerkmalNr': '600002','EinzelteilNr':'9002','Anzahl':'8'},
               {'MerkmalNr': '600002','EinzelteilNr':'9005','Anzahl':'1'},
               {'MerkmalNr': '600003','EinzelteilNr':'6001','Anzahl':'1'},
               {'MerkmalNr': '600003','EinzelteilNr':'6002','Anzahl':'1'},
               {'MerkmalNr': '600003','EinzelteilNr':'9001','Anzahl':'8'},
               {'MerkmalNr': '600003','EinzelteilNr':'9002','Anzahl':'8'},
               {'MerkmalNr': '600003','EinzelteilNr':'9005','Anzahl':'1'},
               {'MerkmalNr': '700001','EinzelteilNr':'7001','Anzahl':'2'},
               {'MerkmalNr': '700001','EinzelteilNr':'7002','Anzahl':'2'},
               {'MerkmalNr': '700001','EinzelteilNr':'7003','Anzahl':'2'},
               {'MerkmalNr': '700001','EinzelteilNr':'9001','Anzahl':'6'},
               {'MerkmalNr': '700001','EinzelteilNr':'9002','Anzahl':'6'},
               {'MerkmalNr': '700002','EinzelteilNr':'7001','Anzahl':'2'},
               {'MerkmalNr': '700002','EinzelteilNr':'7002','Anzahl':'2'},
               {'MerkmalNr': '700002','EinzelteilNr':'7003','Anzahl':'2'},
               {'MerkmalNr': '700002','EinzelteilNr':'9001','Anzahl':'6'},
               {'MerkmalNr': '700002','EinzelteilNr':'9002','Anzahl':'6'},
               {'MerkmalNr': '700003','EinzelteilNr':'7001','Anzahl':'2'},
               {'MerkmalNr': '700003','EinzelteilNr':'7002','Anzahl':'2'},
               {'MerkmalNr': '700003','EinzelteilNr':'7003','Anzahl':'2'},
               {'MerkmalNr': '700003','EinzelteilNr':'9001','Anzahl':'6'},
               {'MerkmalNr': '700003','EinzelteilNr':'9002','Anzahl':'6'},               
               {'MerkmalNr': '800001','EinzelteilNr':'8001','Anzahl':'1'},
               {'MerkmalNr': '800001','EinzelteilNr':'9001','Anzahl':'3'},
               {'MerkmalNr': '800001','EinzelteilNr':'9002','Anzahl':'3'},
               {'MerkmalNr': '800002','EinzelteilNr':'8001','Anzahl':'1'},
               {'MerkmalNr': '800002','EinzelteilNr':'9001','Anzahl':'3'},
               {'MerkmalNr': '800002','EinzelteilNr':'9002','Anzahl':'3'},
               {'MerkmalNr': '800003','EinzelteilNr':'8001','Anzahl':'1'},
               {'MerkmalNr': '800003','EinzelteilNr':'9001','Anzahl':'3'},
               {'MerkmalNr': '800003','EinzelteilNr':'9002','Anzahl':'3'}]
               

my_db = db_communication()
for i in range(len(stueckliste)):
     my_db.insert_data("Stuecklistenteil", stuecklistenteil[i])
my_db.close()

Successfully connected.
INSERT INTO `Stuecklistenteil` (`MerkmalNr`, `EinzelteilNr`, `Anzahl`) VALUES ('100001', '9001', '3');
Rollback, something went wrong.
INSERT INTO `Stuecklistenteil` (`MerkmalNr`, `EinzelteilNr`, `Anzahl`) VALUES ('100001', '9002', '3');
Rollback, something went wrong.
INSERT INTO `Stuecklistenteil` (`MerkmalNr`, `EinzelteilNr`, `Anzahl`) VALUES ('100001', '9004', '45');
Rollback, something went wrong.
INSERT INTO `Stuecklistenteil` (`MerkmalNr`, `EinzelteilNr`, `Anzahl`) VALUES ('100002', '9001', '4');
Rollback, something went wrong.
INSERT INTO `Stuecklistenteil` (`MerkmalNr`, `EinzelteilNr`, `Anzahl`) VALUES ('100002', '9002', '4');
Rollback, something went wrong.
INSERT INTO `Stuecklistenteil` (`MerkmalNr`, `EinzelteilNr`, `Anzahl`) VALUES ('100002', '9004', '55');
Rollback, something went wrong.
INSERT INTO `Stuecklistenteil` (`MerkmalNr`, `EinzelteilNr`, `Anzahl`) VALUES ('200001', '2001', '2');
Rollback, something went wrong.
INSERT INTO `Stuecklistenteil

Rollback, something went wrong.
INSERT INTO `Stuecklistenteil` (`MerkmalNr`, `EinzelteilNr`, `Anzahl`) VALUES ('500003', '5003', '2');
Rollback, something went wrong.
INSERT INTO `Stuecklistenteil` (`MerkmalNr`, `EinzelteilNr`, `Anzahl`) VALUES ('500003', '5004', '1');
Rollback, something went wrong.
INSERT INTO `Stuecklistenteil` (`MerkmalNr`, `EinzelteilNr`, `Anzahl`) VALUES ('500003', '9001', '5');
Rollback, something went wrong.
INSERT INTO `Stuecklistenteil` (`MerkmalNr`, `EinzelteilNr`, `Anzahl`) VALUES ('500003', '9002', '5');
Rollback, something went wrong.
INSERT INTO `Stuecklistenteil` (`MerkmalNr`, `EinzelteilNr`, `Anzahl`) VALUES ('600001', '6001', '1');
Rollback, something went wrong.
INSERT INTO `Stuecklistenteil` (`MerkmalNr`, `EinzelteilNr`, `Anzahl`) VALUES ('600001', '6002', '1');
Rollback, something went wrong.
INSERT INTO `Stuecklistenteil` (`MerkmalNr`, `EinzelteilNr`, `Anzahl`) VALUES ('600001', '9001', '8');
Rollback, something went wrong.
INSERT INTO `Stuecklist

IndexError: list index out of range